## 1.Preparations
### 1.1 Import necessary libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import transformers
import pandas as pd
import os
import random
from pathlib import Path
from easydict import EasyDict as edict
import math
import codecs
import openprompt
from openprompt.data_utils import InputExample
from openprompt.prompts import ManualTemplate,MixedTemplate
from openprompt.plms import load_plm

### 1.2 Hyperparameters

In [2]:
cfg = edict({
    'name': 'movie review',
    'pre_trained': False,
    'num_classes': 2,
    'batch_size': 24,
    'epoch_size': 2,
    'weight_decay': 3e-5,
    'data_path': './data/',
    'device_target': 'Ascend',
    'device_id': 0,
    'keep_checkpoint_max': 1,
    'checkpoint_path': './ckpt/train_textcnn-4_149.ckpt',
    'word_len': 51,
    'vec_length': 40
})


### 1.3 Load data

In [3]:
with open("./data/rt-polarity.neg", 'r', encoding='utf-8') as f:
        print("Negative reivews:")
        for i in range(5):
            print("[{0}]:{1}".format(i,f.readline()))
with open("./data/rt-polarity.pos", 'r', encoding='utf-8') as f:
        print("Positive reivews:")
        for i in range(5):
            print("[{0}]:{1}".format(i,f.readline()))

Negative reivews:
[0]:simplistic , silly and tedious . 

[1]:it's so laddish and juvenile , only teenage boys could possibly find it funny . 

[2]:exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 

[3]:[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . 

[4]:a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . 

Positive reivews:
[0]:the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

[1]:the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth . 

[2]:effective but too-tepid biopic

In [4]:
from torch.utils.data import Dataset, DataLoader
import torch

class Generator:
    def __init__(self, input_list):
        self.input_list = input_list

    def __getitem__(self, item):
        return (
            torch.tensor(self.input_list[item][0], dtype=torch.int32),
            torch.tensor(self.input_list[item][1], dtype=torch.int32)
        )

    def __len__(self):
        return len(self.input_list)

class MovieReview:
    '''
    影评数据集
    '''
    def __init__(self, root_dir, maxlen, split):
        '''
        input:
            root_dir: 影评数据目录
            maxlen: 设置句子最大长度
            split: 设置数据集中训练/评估的比例
        '''
        self.path = root_dir
        self.feelMap = {
            'neg':0,
            'pos':1
        }
        self.files = []

        self.doConvert = False
        
        mypath = Path(self.path)
        if not mypath.exists() or not mypath.is_dir():
            print("please check the root_dir!")
            raise ValueError

        # 在数据目录中找到文件
        for root, _, filename in os.walk(self.path):
            for each in filename:
                self.files.append(os.path.join(root, each))
            break

        # 确认是否为两个文件.neg与.pos
        if len(self.files) != 2:
            print("There are {} files in the root_dir".format(len(self.files)))
            raise ValueError

        # 读取数据
        self.word_num = 0
        self.maxlen = 0
        self.minlen = float("inf")
        self.maxlen = float("-inf")
        self.Pos = []
        self.Neg = []
        for filename in self.files:
            f = codecs.open(filename, 'r')
            ff = f.read()
            file_object = codecs.open(filename, 'w', 'utf-8')
            file_object.write(ff)
            self.read_data(filename)
        self.PosNeg = self.Pos + self.Neg
        self.split_dataset(split=split)

    def read_data(self, filePath):
        with open(filePath,'r') as f:
            for sentence in f.readlines():
                sentence = sentence.replace('\n','')\
                                    .replace('"','')\
                                    .replace('\'','')\
                                    .replace('.','')\
                                    .replace(',','')\
                                    .replace('[','')\
                                    .replace(']','')\
                                    .replace('(','')\
                                    .replace(')','')\
                                    .replace(':','')\
                                    .replace('--','')\
                                    .replace('-',' ')\
                                    .replace('\\','')\
                                    .replace('0','')\
                                    .replace('1','')\
                                    .replace('2','')\
                                    .replace('3','')\
                                    .replace('4','')\
                                    .replace('5','')\
                                    .replace('6','')\
                                    .replace('7','')\
                                    .replace('8','')\
                                    .replace('9','')\
                                    .replace('`','')\
                                    .replace('=','')\
                                    .replace('$','')\
                                    .replace('/','')\
                                    .replace('*','')\
                                    .replace(';','')\
                                    .replace('<b>','')\
                                    .replace('%','')
                #sentence = sentence.split(' ')
                #sentence = list(filter(lambda x: x, sentence))
                if sentence:
                    self.word_num += len(sentence)
                    self.maxlen = self.maxlen if self.maxlen >= len(sentence) else len(sentence)
                    self.minlen = self.minlen if self.minlen <= len(sentence) else len(sentence)
                    if 'pos' in filePath:
                        self.Pos.append([sentence,self.feelMap['pos']])
                    else:
                        self.Neg.append([sentence,self.feelMap['neg']])


    def split_dataset(self, split):
        '''
        分割为训练集与测试集

        '''
        trunk_pos_size = math.ceil((1-split)*len(self.Pos))
        trunk_neg_size = math.ceil((1-split)*len(self.Neg))
        trunk_num = int(1/(1-split))
        pos_temp=list()
        neg_temp=list()
        for index in range(trunk_num):
            pos_temp.append(self.Pos[index*trunk_pos_size:(index+1)*trunk_pos_size])
            neg_temp.append(self.Neg[index*trunk_neg_size:(index+1)*trunk_neg_size])
        self.test = pos_temp.pop(2)+neg_temp.pop(2)
        self.train = [i for item in pos_temp+neg_temp for i in item]
        random.shuffle(self.train)
        # random.shuffle(self.test)

    def get_dict_len(self):
        '''
        获得数据集中文字组成的词典长度
        '''
        if self.doConvert:
            return len(self.Vocab)
        else:
            print("Haven't finished Text2Vec")
            return -1

    def create_train_dataset(self, batch_size, epoch_size):
        train_examples = []
        for i, (text, label) in enumerate(self.train):
            # 创建 InputExample 对象并添加到列表中
            id = str(i)
            example = InputExample(guid=id, text_a=text, label=label)
            train_examples.append(example)
        return train_examples
    
    def create_test_dataset(self, batch_size):
        test_examples = []
        for i, (text, label) in enumerate(self.test):
            # 创建 InputExample 对象并添加到列表中
            #print("text:", text)
            id = str(i)
            example = InputExample(guid=id, text_a=text, label=label)
            test_examples.append(example)
        return test_examples



In [5]:
instance = MovieReview(root_dir=cfg.data_path, maxlen=cfg.word_len, split=0.9)
dataset = instance.create_train_dataset(batch_size=cfg.batch_size,epoch_size=cfg.epoch_size)
testset = instance.create_test_dataset(batch_size=cfg.batch_size)
for i in range(5):
    print("Trainset{0}:{1}".format(i,dataset[i]))
    print("Testset{0}:{1}".format(i,testset[i]))

Trainset0:{
  "guid": "0",
  "label": 1,
  "meta": {},
  "text_a": "an impressive hybrid  ",
  "text_b": "",
  "tgt_text": null
}

Testset0:{
  "guid": "0",
  "label": 1,
  "meta": {},
  "text_a": "a deep and meaningful film  ",
  "text_b": "",
  "tgt_text": null
}

Trainset1:{
  "guid": "1",
  "label": 0,
  "meta": {},
  "text_a": "with a tone as variable as the cinematography  schaeffers film never settles into the light footed enchantment the material needs  and the characters quirks and foibles never jell into charm  ",
  "text_b": "",
  "tgt_text": null
}

Testset1:{
  "guid": "1",
  "label": 1,
  "meta": {},
  "text_a": "the films welcome breeziness and some unbelievably hilarious moments  most portraying the idiocy of the film industry  make it mostly worth the trip  ",
  "text_b": "",
  "tgt_text": null
}

Trainset2:{
  "guid": "2",
  "label": 1,
  "meta": {},
  "text_a": "an intelligent  multi layered and profoundly humanist  not to mention gently political  meditation on the 

## 2.Prompting 

### Step 1. Define a task
The first step is to determine the current NLP task, think about what’s your data looks like and what do you want from the data! That is, the essence of this step is to determine the classses and the InputExample of the task. For simplicity, we use Sentiment Analysis as an example.

You can also use our pre-defined Data Processors to get train/dev/test dataset for a given task.

In [6]:
classes = [ # There are two classes in Sentiment Analysis, one for negative and one for positive
    "negative",
    "positive"
]

### Step 2. Obtain a PLM

Choose a PLM to support your task. Different models have different attributes, we encourge you to use OpenPrompt to explore the potential of various PLMs. OpenPrompt is compatible with models on huggingface, the following models have been tested:

Masked Language Models (MLM): BERT, RoBERTa, ALBERT

Autoregressive Language Models (LM): GPT, GPT2

Sequence-to-Sequence Models (Seq2Seq): T5

Simply use a get_model_class to obtain your PLM.

In [7]:
plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2")

/Users/lily/anaconda3/envs/test1/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Step 3. Define a Template

A Template is a modifier of the original input text, which is also one of the most important modules in prompt-learning. A more advanced tutorial to define a template is in How to Write a Template?

Here is an example, where the <text_a> will be replaced by the text_a in InputExample, and the <mask> will be used to predict a label word.

In [9]:
promptTemplate = MixedTemplate(
    model=plm,
    text='{"placeholder": "text_a"}{"soft":"The emotional tendency of the comment of the movie is"}{"mask"}',
    tokenizer=tokenizer,
)

### Step 4. Define a Verbalizer

A Verbalizer is another important (but not necessary such as in generation) in prompt-learning,which projects the original labels (we have defined them as classes, remember?) to a set of label words. 

Here is an example that we project the negative class to the word bad project the positive class to the words good, wonderful, great.

In [10]:
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    num_classes = len(classes),
    label_words = {
        "negative": ["bad", ],
        "positive": ["good", "wonderful", "great"]
    },
    tokenizer = tokenizer,
)

In [11]:
print(f'input example: \n {dataset[0]}')
wrapped_example = promptTemplate.wrap_one_example(dataset[0])
print(f'wrapped example:')
for ele in wrapped_example[0]:
    print(ele)

input example: 
 {
  "guid": "0",
  "label": 1,
  "meta": {},
  "text_a": "an impressive hybrid  ",
  "text_b": "",
  "tgt_text": null
}

wrapped example:
{'text': 'an impressive hybrid  ', 'soft_token_ids': 0, 'loss_ids': 0, 'shortenable_ids': 1}
{'text': 'The', 'soft_token_ids': 1, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġemotional', 'soft_token_ids': 2, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġtendency', 'soft_token_ids': 3, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġof', 'soft_token_ids': 4, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġthe', 'soft_token_ids': 5, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġcomment', 'soft_token_ids': 6, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġof', 'soft_token_ids': 7, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġthe', 'soft_token_ids': 8, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġmovie', 'soft_token_ids': 9, 'loss_ids': 0, 'shortenable_ids': 0}
{'text': 'Ġis', 'soft_token_ids': 10, 'loss_ids': 0, 'shortenab

In [12]:
wrap_tokenizer = WrapperClass(max_seq_length=32, tokenizer=tokenizer)
tokenized_example = wrap_tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)

In [13]:
print(f'tokenized example: \n {tokenized_example}\n')

print('printing each key-value pair:')
for key, value in tokenized_example.items():
    print(f'{key}:{value}')

# input_ids 变回 tokens
print('\ninput_ids to tokens:')
tokens = []   
for id in tokenized_example['input_ids']:
    tokens.append(tokenizer.convert_ids_to_tokens(id))

print(' '.join(tokens))

tokenized example: 
 {'input_ids': [272, 8036, 14554, 220, 220, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257], 'soft_token_ids': [0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'loss_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids_len': 15}

printing each key-value pair:
input_ids:[272, 8036, 14554, 220, 220, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257]
soft_token_ids:[0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
loss_ids:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Step 5. Construct a PromptModel

Given the task, now we have a PLM, a Template and a Verbalizer, we combine them into a PromptModel.

Note that although this example naively combine the three modules, you can actually define some complicated interactions among them.

In [14]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

### Step 6. Define a DataLoader

A PromptDataLoader is basically a prompt version of pytorch Dataloader, which also includes a Tokenizer and a Template.

In [15]:
from openprompt import PromptDataLoader
train_data_loader = PromptDataLoader(
    dataset= dataset, 
    template=promptTemplate, 
    tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, 
    batch_size=cfg.batch_size,
    shuffle=True, 
    )


tokenizing: 9594it [00:03, 2927.95it/s]


### Train and Inference


In [16]:
from torch.optim import Adam
import torch.nn.functional as F

device = torch.device("mps:0")
#device = torch.device("cpu")
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn.to(device)
optimizer = Adam(promptModel.parameters(), lr=1e-5, weight_decay=cfg.weight_decay)
optimizer.zero_grad()
epoch_size = cfg.epoch_size
num_train = 20
it = 0

for epoch in range(epoch_size):
    for batch_idx, data in enumerate(train_data_loader):
        it += 1
        if it > num_train:
            break
        labels = data["label"]
        logits = promptModel(data)

        logits.to(device)
        labels.to(device)
        loss = loss_fn(logits, labels)
        print(f'Epoch [{epoch+1}/{epoch_size}], Batch [{batch_idx+1}/{len(train_data_loader)}], Loss: {loss:.4f}')
        loss.to(device)
        # Backward pass
        loss.backward(retain_graph=True)
        # Update weights
        optimizer.step()
        optimizer.zero_grad()
        
        logits.to(torch.device('cpu'))
        labels.to(torch.device('cpu'))
      

Epoch [1/2], Batch [1/400], Loss: 0.5854
Epoch [1/2], Batch [2/400], Loss: 0.5201
Epoch [1/2], Batch [3/400], Loss: 0.5907
Epoch [1/2], Batch [4/400], Loss: 0.5395
Epoch [1/2], Batch [5/400], Loss: 0.4900
Epoch [1/2], Batch [6/400], Loss: 0.5224
Epoch [1/2], Batch [7/400], Loss: 0.5658
Epoch [1/2], Batch [8/400], Loss: 0.4019
Epoch [1/2], Batch [9/400], Loss: 0.3046
Epoch [1/2], Batch [10/400], Loss: 0.6805
Epoch [1/2], Batch [11/400], Loss: 0.4725
Epoch [1/2], Batch [12/400], Loss: 0.5130
Epoch [1/2], Batch [13/400], Loss: 0.4627
Epoch [1/2], Batch [14/400], Loss: 0.4290
Epoch [1/2], Batch [15/400], Loss: 0.4843
Epoch [1/2], Batch [16/400], Loss: 0.4302
Epoch [1/2], Batch [17/400], Loss: 0.2591
Epoch [1/2], Batch [18/400], Loss: 0.3817
Epoch [1/2], Batch [19/400], Loss: 0.1791
Epoch [1/2], Batch [20/400], Loss: 0.1887
Epoch [1/2], Batch [21/400], Loss: 0.2459
Epoch [1/2], Batch [22/400], Loss: 0.5719
Epoch [1/2], Batch [23/400], Loss: 0.5541
Epoch [1/2], Batch [24/400], Loss: 0.5779
E

In [24]:
torch.save(promptModel.state_dict(), cfg.checkpoint_path)

### Testing

* In this case , we first use some test to briefly test the performance of the model, and then use the model to predict the sentiment of the given text.

#### Simple Cases

In [25]:
testset = [ # For simplicity, there's only two examples
    # text_a is the input text of the data, some other datasets may have multiple input sentences in one example.
    InputExample(
        guid = 1,
        text_a = "The film was badly made.",
        label = 0
    ),
    InputExample(
        guid = 2,
        text_a = "The film was very good!!",
        label = 1
    ),
    InputExample(
        guid = 3,
        text_a = "I love this movie.",
        label = 1
    ),
    InputExample(
        guid = 4,
        text_a = "One of the worst movies I've ever seen.",
        label = 0
    ),
    
    InputExample(
        guid = 5,
        text_a = "Just terrible.",
        label = 0
    ),
    InputExample(
        guid = 6,
        text_a = "Although the plot is a bit cliched, the film is still very good.",
        label = 1
    ),
    InputExample(
        guid = 7,
        text_a = "The film is not as good as I expected.",
        label = 0
    ),
    InputExample(
        guid = 8,
        text_a = "Maybe I'm too picky, but I don't think the film is good.",
        label = 0
    ),
]

test_data_loader = PromptDataLoader(
    dataset = testset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    drop_last = True,
)

tokenizing: 8it [00:00, 1456.42it/s]


In [26]:
promptModel.eval()


PromptForClassification(
  (prompt_model): PromptModel(
    (plm): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50258, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Conv1D()
              (c_proj): Conv1D()
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.1, inplace=False)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): GPT2MLP(
              (c_fc): Conv1D()
              (c_proj): Conv1D()
              (act): NewGELUActivation()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
        )
        (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (lm_head): Lin

In [27]:
# making zero-shot inference using pretrained MLM with prompt
num = len(testset)
num_right = 0
with torch.no_grad():
    for batch in test_data_loader:
        logits = promptModel(batch)
        right_class = batch['label']
        preds = torch.argmax(logits, dim = -1)
        print(classes[preds])
        if preds == right_class:
            num_right += 1
        else:
            print("Wrong prediction for the following sentece:",end=' ')
            print(batch['text_a'])
# predictions would be 1, 0 for classes 'positive', 'negative'
print(f'Accuracy: {num_right/num:.4f}')

negative
positive
positive
negative
negative
positive
negative
negative
Accuracy: 1.0000


#### Test on Large Dataset

In [28]:
testset = instance.create_test_dataset(batch_size=cfg.batch_size)
for i in range(5):
    print("Trainset{0}:{1}".format(i,dataset[i]))
    print("Testset{0}:{1}".format(i,testset[i]))

Trainset0:{
  "guid": "0",
  "label": 1,
  "meta": {},
  "text_a": "an impressive hybrid  ",
  "text_b": "",
  "tgt_text": null
}

Testset0:{
  "guid": "0",
  "label": 1,
  "meta": {},
  "text_a": "a deep and meaningful film  ",
  "text_b": "",
  "tgt_text": null
}

Trainset1:{
  "guid": "1",
  "label": 0,
  "meta": {},
  "text_a": "with a tone as variable as the cinematography  schaeffers film never settles into the light footed enchantment the material needs  and the characters quirks and foibles never jell into charm  ",
  "text_b": "",
  "tgt_text": null
}

Testset1:{
  "guid": "1",
  "label": 1,
  "meta": {},
  "text_a": "the films welcome breeziness and some unbelievably hilarious moments  most portraying the idiocy of the film industry  make it mostly worth the trip  ",
  "text_b": "",
  "tgt_text": null
}

Trainset2:{
  "guid": "2",
  "label": 1,
  "meta": {},
  "text_a": "an intelligent  multi layered and profoundly humanist  not to mention gently political  meditation on the 

In [31]:
test_data_loader = PromptDataLoader(
    dataset = testset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    drop_last = True,
    shuffle=True
)

tokenizing: 1067it [00:00, 1958.88it/s]


In [34]:
promptModel.eval()
# making zero-shot inference using pretrained MLM with prompt
num = 0
num_right = 0
with torch.no_grad():
    for batch in test_data_loader:
        num+=1
        logits = promptModel(batch)
        right_class = batch['label']
        preds = torch.argmax(logits, dim = -1)
        print(classes[preds])
        if preds == right_class:
            num_right += 1
        print ("Accuracy: ", num_right/num)


positive
Accuracy:  1.0
positive
Accuracy:  1.0
positive
Accuracy:  1.0
positive
Accuracy:  1.0
positive
Accuracy:  1.0
positive
Accuracy:  1.0
negative
Accuracy:  1.0
negative
Accuracy:  1.0
positive
Accuracy:  1.0
positive
Accuracy:  1.0
negative
Accuracy:  1.0
negative
Accuracy:  0.9166666666666666
negative
Accuracy:  0.9230769230769231
positive
Accuracy:  0.9285714285714286
negative
Accuracy:  0.9333333333333333
positive
Accuracy:  0.9375
positive
Accuracy:  0.9411764705882353
negative
Accuracy:  0.9444444444444444
positive
Accuracy:  0.9473684210526315
negative
Accuracy:  0.95
positive
Accuracy:  0.9523809523809523
positive
Accuracy:  0.9090909090909091
negative
Accuracy:  0.9130434782608695
negative
Accuracy:  0.9166666666666666
positive
Accuracy:  0.92
negative
Accuracy:  0.9230769230769231
negative
Accuracy:  0.9259259259259259
negative
Accuracy:  0.9285714285714286
positive
Accuracy:  0.9310344827586207
positive
Accuracy:  0.9333333333333333
negative
Accuracy:  0.9032258064516

#### Final Accuracy

In [35]:
print(f'Accuracy: {num_right/num:.4f}')

Accuracy: 0.8229
